In [ ]:
import os
os.environ["JOBLIB_MULTIPROCESSING_START_METHOD"] = "spawn"

In [ ]:
from pathlib import Path
from ops.ecris.analysis.io import read_csd_from_file_pair

CSD_FILEPATHS = list(Path('./data/csds').glob('csd_*'))
FILES_START: int = 180
N_FILES: int = 100
csds = [read_csd_from_file_pair(file) for file in CSD_FILEPATHS[FILES_START:FILES_START + N_FILES]]
print(f'Loaded {len(csds)} of {len(CSD_FILEPATHS)} CSD files.')

In [ ]:
from ops.ecris.analysis.csd.m_over_q import estimate_m_over_q
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
import numpy as np
from itertools import permutations
import random
from IPython.display import clear_output

from ops.ecris.analysis.model.csd import CSD
from ops.ecris.analysis.model.element import Element
from ops.ecris.analysis.csd.peaks import find_element_peaks

def sorted_permutations(to_permute):
    return [list(p) for p in permutations(to_permute, r=7) if list(p) == sorted(p)] 

def get_training_peaks(csd: CSD):
    clear_output(wait=True)
    csd.m_over_q = estimate_m_over_q(csd)
    oxygen = Element('Oxygen', 'O', 16, 8)
    o2_peaks = find_element_peaks(csd, oxygen, 0.2)
    all_peaks, properties = find_peaks(csd.beam_current, prominence=0.1)
    sorted_peaks = np.flip(np.argsort(properties['prominences']))
    plt.plot(csd.m_over_q, csd.beam_current, '--')
    plt.plot(csd.m_over_q[o2_peaks.indexes], o2_peaks.beam_current, '+')
    for i in range(1, 9):
        plt.axvline(16/i, ls='--', alpha=0.25, c='k')
    plt.grid()
    plt.show()
    accept = input("Accept peaks? (Y/n)")
    if accept == 'n':
        return []


    for highest in [10, 15]:
        highest_peaks = sorted(all_peaks[sorted_peaks][:highest])
        permutations = sorted_permutations([int(v) for v in highest_peaks])
        total_sets = len(permutations)
        for i, p in enumerate(permutations):
            if p == o2_peaks.indexes:
                correct_peaks = permutations.pop(i)
                break
        if len(permutations) != total_sets - 1:
            print(f'Failed to fit with {highest=} with {len(permutations)=}')
            continue
        if len(permutations) > 1:
            bad_peaks = [permutations[i] for i in [random.randint(0, len(permutations) - 1) for _ in range(49)]]
        elif len(permutations) == 1:
            bad_peaks = permutations[0]
        else:
            bad_peaks = []
        training_peaks = [correct_peaks] + bad_peaks
        return training_peaks
    return []

In [ ]:
x = []
y = []
failed = 0
for csd in csds:
    training_peaks = get_training_peaks(csd)
    if not training_peaks:
        failed += 1
        continue
    x.extend(training_peaks)
    y.extend([1] + (len(training_peaks) - 1)*[0])
print(f'{failed=}/{len(csds)}')

In [ ]:
x_array = np.array(x)
y_array = np.array(y)
print(x_array.shape, y_array.shape)
x_training = np.load('x_training.npy')
y_training = np.load('y_training.npy')
print(x_training.shape, y_training.shape)
x_output = np.concatenate([x_training, x_array])
y_output = np.concatenate([y_training, y_array])
print(x_output.shape, y_output.shape)

In [ ]:
np.save('x_training.bak', x_training)
np.save('y_training.bak', y_training)
np.save('x_training', x_output)
np.save('y_training', y_output)

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, f1_score, recall_score, roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import StratifiedKFold, HalvingGridSearchCV
from sklearn.utils import resample
import numpy as np

X_raw = np.load('x_training.npy')
Y_raw = np.load('y_training.npy')

def balance_dataset(X, y, random_state=42):
    # Separate classes
    X1, y1 = X[y == 1], y[y == 1]          # minority (oxygen)
    X0, y0 = X[y == 0], y[y == 0]          # majority (non‑oxygen)

    # Down‑sample majority to the size of minority (or a chosen ratio)
    X0_down, y0_down = resample(
        X0, y0,
        replace=False,
        n_samples=int(len(X1) * 3),   # 1:3 ratio (tune as needed)
        random_state=random_state)

    X_bal = np.vstack([X1, X0_down])
    y_bal = np.hstack([y1, y0_down])
    return X_bal, y_bal

X, Y = balance_dataset(X_raw, Y_raw)

pipeline = Pipeline([('scaler', StandardScaler()), ('mlp', MLPClassifier(random_state=42, 
                                                                         max_iter=500, 
                                                                         early_stopping=True, 
                                                                         n_iter_no_change=10, 
                                                                         verbose=False))
                                                                         ])
param_grid = {
    'mlp__hidden_layer_sizes': [
        (5,), (7,), (10,), (14,), (20,), (30,), (50,), (100,),
        (10, 5), (14, 7), (20, 10), (30, 15), (50, 25), (100, 50),
        (20, 10, 5), (30, 15, 7), (50, 25, 10), (100, 50, 20)
    ],
    'mlp__activation': ['relu', 'tanh'],
    'mlp__solver': ['adam', 'sgd'],
    'mlp__alpha': [1e-5, 1e-4, 1e-3],
    'mlp__learning_rate_init': [1e-4, 1e-3, 1e-2],
    'mlp__batch_size': [32, 64, 128],
    'mlp__learning_rate': ['constant', 'adaptive'],
    'mlp__validation_fraction': [0.1, 0.2],
    'mlp__n_iter_no_change': [10, 20],
}

scorer = make_scorer(f1_score, pos_label=1)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

grid = HalvingGridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    scoring=scorer,
    cv=cv,
    n_jobs=-1,               
    verbose=2,               
    refit=True,              #
    return_train_score=True
)
grid.fit(X, Y)

## Save best model

In [ ]:
import joblib
import json
best_pipe = grid.best_estimator_
params = grid.best_params_
model_path = Path('mlp_csd_oxygen.pkl')
params_path = Path('mlp_csd_oxygen_params.json')
joblib.dump(best_pipe, model_path)
with open(params_path, 'w') as f:
    json.dump(params, f, indent=2, sort_keys=True)

## Use Model

In [ ]:
import joblib
clf = joblib.load('mlp_csd_oxygen.pkl')

def estimate_correct_peaks(csd):
    all_peaks, properties = find_peaks(csd.beam_current, prominence=0.1, height=(10, None))
    sorted_peaks = np.flip(np.argsort(properties['prominences']))
    highest_peaks = sorted(all_peaks[sorted_peaks][:10])
    sorted_peaks = sorted_permutations([int(v) for v in highest_peaks])
    return sorted_peaks, clf.predict_proba(sorted_peaks)[:, 1]

In [ ]:
CSD_FILEPATHS = list(Path('./data/csds').glob('csd_*'))
FILE_NUM = random.randint(0, len(CSD_FILEPATHS))
csd = read_csd_from_file_pair(CSD_FILEPATHS[FILE_NUM])
estimated_peaks, probabilities = estimate_correct_peaks(csd)
highest_prob = np.flip(np.argsort(probabilities))[0]
csd.m_over_q = estimate_m_over_q(csd)

plt.plot(csd.m_over_q, csd.beam_current, '--')

highest_peaks = estimated_peaks[highest_prob]
plt.plot(csd.m_over_q[highest_peaks], csd.beam_current[highest_peaks], '+')
for i in range(1, 9):
    plt.axvline(16/i, ls='--', alpha=0.25, c='k')
plt.xlabel('M/Q (Estimated)')
plt.ylabel(r'Beam current ($\mu$A)')
plt.xlim([0,10])
plt.grid()
plt.show()
print(f'Estimated probability of O2 detection: {probabilities[highest_prob]}')

In [ ]:
print(np.flip(np.argsort(probabilities[:,1])))